In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets

In [3]:
path = os.path.join(os.getcwd(),'data')
dfprod = pd.read_csv(os.path.join(path,'produccin-de-pozos-de-gas-y-petrleo-no-convencional.csv'), decimal='.')
dffrac = pd.read_csv(os.path.join(path,'datos-de-fractura-de-pozos-de-hidrocarburos-adjunto-iv-actualizacin-diaria.csv'), decimal='.')
dfprod['data'] = dfprod['fecha_data'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
dfprod['qo(kbpd)'] = 6.29*dfprod['prod_pet']/dfprod['fecha_data'].apply(lambda x:int(x[-2:]))
dfprod['qg(km3d)'] = dfprod['prod_gas']/dfprod['fecha_data'].apply(lambda x:int(x[-2:]))

In [4]:
dfprod = dfprod[dfprod['formacion']=='vaca muerta']
dfprod = dfprod[dfprod['tipoestado']!='Parado Transitoriamente']

In [5]:
dfprod2 = dfprod.drop([
    'anio',
    'mes',
    'iny_agua',
    'iny_gas',
    'iny_co2',
    'iny_otro',
    'tef',
    'vida_util',
    'tipoextraccion',
    'tipoestado',
    'tipopozo',
    'observaciones',
    'fechaingreso',
    'rectificado',
    'habilitado',
    'idusuario',
    'empresa',
    'sigla',
    'formprod',
    'profundidad',
    'formacion',
    'idareapermisoconcesion',
    'areapermisoconcesion',
    'idareayacimiento',
    'cuenca',
    'provincia',
    'coordenadax',
    'coordenaday',
    'tipo_de_recurso',
    'proyecto',
    'clasificacion',
    'subclasificacion',
    'sub_tipo_recurso',
    'fecha_data',
], axis=1)

In [6]:
idp = 162180
dfprod2[dfprod2['idpozo']==idp].sort_values('data').head()

,idempresa,idpozo,prod_pet,prod_gas,prod_agua,areayacimiento,data,qo(kbpd),qg(km3d)
224539,PLU,162180,2126.54,2599.363,2685.12,LA CALERA,2021-06-30,445.864553,86.645433
226858,PLU,162180,4136.28,5809.501,2532.65,LA CALERA,2021-07-31,839.264555,187.403258
226983,PLU,162180,4688.76,6864.183,2226.42,LA CALERA,2021-08-31,951.364529,221.425258
229317,PLU,162180,4070.19,6301.298,1478.81,LA CALERA,2021-09-30,853.383170,210.043267
230686,PLU,162180,4030.17,6187.724,1543.32,LA CALERA,2021-10-31,817.734494,199.604000


In [7]:
datazero = dfprod2.groupby('idpozo')['data'].min()

In [8]:
dfprod3 = dfprod2.merge(right=datazero, how='left', on='idpozo', suffixes=('', '_start'))

dfprod3.head()

,idempresa,idpozo,prod_pet,prod_gas,prod_agua,areayacimiento,data,qo(kbpd),qg(km3d),data_start
0,YSUR,145329,0.0,0.0,0.0,CORTADERA,2018-01-31,0.0,0.0,2013-07-31
1,YSUR,145329,0.0,0.0,0.0,CORTADERA,2017-01-31,0.0,0.0,2013-07-31
2,YSUR,145330,0.0,0.0,0.0,HUACALERA,2015-01-31,0.0,0.0,2013-07-31
3,YSUR,145330,0.0,0.0,0.0,HUACALERA,2018-01-31,0.0,0.0,2013-07-31
4,YSUR,145330,0.0,0.0,0.0,HUACALERA,2017-01-31,0.0,0.0,2013-07-31


In [9]:
dfprod3['m'] = dfprod3['data'].dt.to_period('M').astype('int64') - dfprod3['data_start'].dt.to_period('M').astype('int64')

dfprod3[dfprod3['idpozo']==idp].sort_values('data').head()

,idempresa,idpozo,prod_pet,prod_gas,prod_agua,areayacimiento,data,qo(kbpd),qg(km3d),data_start,m
119965,PLU,162180,2126.54,2599.363,2685.12,LA CALERA,2021-06-30,445.864553,86.645433,2021-06-30,0
120388,PLU,162180,4136.28,5809.501,2532.65,LA CALERA,2021-07-31,839.264555,187.403258,2021-06-30,1
120430,PLU,162180,4688.76,6864.183,2226.42,LA CALERA,2021-08-31,951.364529,221.425258,2021-06-30,2
120794,PLU,162180,4070.19,6301.298,1478.81,LA CALERA,2021-09-30,853.383170,210.043267,2021-06-30,3
120999,PLU,162180,4030.17,6187.724,1543.32,LA CALERA,2021-10-31,817.734494,199.604000,2021-06-30,4


In [11]:
dffrac.columns

Index(['id_base_fractura_adjiv', 'idpozo', 'sigla', 'cuenca',
       'areapermisoconcesion', 'yacimiento', 'formacion_productiva',
       'tipo_reservorio', 'subtipo_reservorio', 'longitud_rama_horizontal_m',
       'cantidad_fracturas', 'tipo_terminacion', 'arena_bombeada_nacional_tn',
       'arena_bombeada_importada_tn', 'agua_inyectada_m3', 'co2_inyectado_m3',
       'presion_maxima_psi', 'potencia_equipos_fractura_hp',
       'fecha_inicio_fractura', 'fecha_fin_fractura', 'fecha_data', 'anio_if',
       'mes_if', 'anio_ff', 'mes_ff', 'anio_carga', 'mes_carga',
       'empresa_informante', 'mes', 'anio'],
      dtype='object')

In [17]:
dfprod4 = dfprod3.merge(right=dffrac[['idpozo','longitud_rama_horizontal_m','cantidad_fracturas']], how='left', on='idpozo')
cols = [
'idempresa',
'idpozo',
# 'prod_pet',
# 'prod_gas',
# 'prod_agua',
'areayacimiento',
'data',
'qo(kbpd)',
'qg(km3d)',
'data_start',
'm',
'longitud_rama_horizontal_m',
'cantidad_fracturas',
]

dfprod4 = dfprod4[cols]
dfprod4

,idempresa,idpozo,areayacimiento,data,qo(kbpd),qg(km3d),data_start,m,longitud_rama_horizontal_m,cantidad_fracturas
0,YSUR,145329,CORTADERA,2018-01-31,0.000000,0.000000,2013-07-31,54,NaN,NaN
1,YSUR,145329,CORTADERA,2017-01-31,0.000000,0.000000,2013-07-31,42,NaN,NaN
2,YSUR,145330,HUACALERA,2015-01-31,0.000000,0.000000,2013-07-31,18,NaN,NaN
3,YSUR,145330,HUACALERA,2018-01-31,0.000000,0.000000,2013-07-31,54,NaN,NaN
4,YSUR,145330,HUACALERA,2017-01-31,0.000000,0.000000,2013-07-31,42,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
135828,ACO,3640,EL CARACOL,2024-06-30,5.310976,0.158560,2006-02-28,220,NaN,NaN
135829,ACO,3640,EL CARACOL,2023-07-31,5.884565,0.302853,2006-02-28,209,NaN,NaN
135830,ACO,3640,EL CARACOL,2024-07-31,5.397611,0.208688,2006-02-28,221,NaN,NaN
135831,ACO,3640,EL CARACOL,2023-08-31,6.296665,0.299045,2006-02-28,210,NaN,NaN
